In [2]:
import pandas as pd
import json
def load_config(path):
    with open(path, 'r') as file:
        return json.load(file)

config_path = './pandas_config.json'
config = load_config(config_path)
parquet_config = config['parquet']

# Setting up the engine as a global constant
ENGINE = parquet_config['engine']
COMPRESSION = parquet_config['compression']
FILE_PATH = "./data/data_parquet"
CONTINENTS = ["asia", "america", "europe"]

In [3]:
df = pd.read_parquet(FILE_PATH + "/processed_data_combined.parquet", engine=ENGINE)


In [4]:
filtered_actions = df[df['action_name']  == ('UpdateEstimatesAction' or"CreateZFWMessageAction")]

In [5]:
flight_numbers = filtered_actions["flight_number"].values
values = filtered_actions["entry_details"].values
action_names = filtered_actions["action_name"].values

In [6]:
import re
def CreateZFWMessageAction(dictionary, string, flight):
    pattern = r"^ZFW (\d+) KG"

    # Using list comprehension to extract digits from matching patterns
    estimated_zfw = re.match(pattern, string).group(1)
    pattern = r"<actualZFW>\s*(\d+)\s*</actualZFW>"
    actual_zfw = re.search(pattern, string).group(1)

    if flight in list(dictionary.keys()):
        dictionary[flight] = {"estimated_zfw": [estimated_zfw], "actual_zfw": [actual_zfw]}
    else:
        dictionary[flight]["estimated_zfw"].append(estimated_zfw)
        dictionary[flight]["actual_zfw"].append(actual_zfw)
    return dictionary

def UpdateEstimatesAction(dictionary, string, flight):
    # Updated pattern to better handle numbers with and without decimal points
    pattern = r"EZFW\s*:\s*(\d+(?:\.\d*)?)\s*KG"

    match = re.search(pattern, string)
    if match:
        estimate_zfw = int(float(match.group(1)))
        if flight in list(dictionary.keys()):
            dictionary[flight]["estimated_zfw"].append(estimate_zfw)
        else:
            dictionary[flight] = {"estimated_zfw": [estimate_zfw]}        
    return dictionary



In [7]:
global final 
final = {}
for flight in flight_numbers:
    subset_df = filtered_actions[filtered_actions["flight_number"] == flight]
    values = subset_df["entry_details"].values
    action_names = subset_df["action_name"].values
    for value, action_name in zip(values, action_names):
        if action_name == "CreateZFWMessageAction":
            returns = CreateZFWMessageAction(final,value, flight)
            final = returns
        else:
            returns = UpdateEstimatesAction(final, value, flight)
            final = returns

In [22]:
filtered_actions[filtered_actions["flight_number"]=="d7918177276d8af"]

,id,creation_time,formatted_creation_time,airline_code,flight_date,departure_airport,departure_city,departure_country,departure_lat,departure_lon,user_name,action_name,stepID,action_mode,log_level,entry_details,flight_number
612963,138100675,2024-05-03 16:54:13,03-05-2024 16:54:13,MN,3,ORD,Chicago,US,41.97694,-87.90815,human,UpdateEstimatesAction,d7918177276d8af,Received,INFO,com.systemone.lc2.estimateshandling.dto.Estima...,d7918177276d8af
612964,138100680,2024-05-03 16:54:13,03-05-2024 16:54:13,MN,3,ORD,Chicago,US,41.97694,-87.90815,human,UpdateEstimatesAction,d7918177276d8af,Saved,INFO,Pax Weight = 24880.0 KG Bag Weight = 3885.0 KG...,d7918177276d8af


In [10]:
testdf = pd.DataFrame.from_dict(final, orient='index')